# Segmentation 3D

## 0. Imports

In [1]:
import cv2

import openalea.phenomenal.display as phm_display
import openalea.phenomenal.data as phm_data
import openalea.phenomenal.display.notebook as phm_display_notebook
import openalea.phenomenal.object as phm_obj
import openalea.phenomenal.multi_view_reconstruction as phm_mvr
import openalea.phenomenal.segmentation as phm_seg

## 1. Prerequisites

### 1.1 Load cloud point of voxel centers

In [2]:
plant_number = 1
voxels_size = 16

bin_images = phm_data.bin_images(plant_number=plant_number)
calibrations = phm_data.calibrations(plant_number=plant_number)
voxel_grid = phm_data.voxel_grid(plant_number=plant_number, voxels_size=voxels_size)

### 1.2 Display voxel grid

In [3]:
phm_display_notebook.show_voxel_grid(voxel_grid)

A Jupyter Widget

## 2. Skeletonization

### 2.1 Compute Graph and the skeleton
Compute the skeleton of the plant voxelgrid

In [4]:
voxel_graph = phm_seg.voxel_graph_from_voxel_grid(
    voxel_grid, 
    connect_all_point=True)

voxel_skeleton = phm_seg.skeletonize(
    voxel_graph.graph,
    voxel_graph.voxels_size,
    subgraph=None)

### 2.2 Display it

In [5]:
phm_display_notebook.show_skeleton(voxel_skeleton)

A Jupyter Widget

### 2.3 Remove not enouth visible segment

In [6]:
# Select images
image_views = list()
for angle in bin_images["side"]:
    projection = calibrations["side"].get_projection(angle)
    image_views.append(phm_obj.ImageView(bin_images["side"][angle], projection))

voxel_skeleton_denoised = phm_seg.segment_reduction(
    voxel_skeleton, image_views, tolerance=4, nb_min_pixel=100)

### 2.4 Display it

In [7]:
phm_display_notebook.show_skeleton(voxel_skeleton_denoised)

A Jupyter Widget

## 3. Maize Segmentation

### 3.1 Process

Voxel skeleton are segmented in several label "stem", "mature_leaf", "growing_leaf" and "unknow"

In [8]:
vms = phm_seg.maize_segmentation(voxel_skeleton_denoised, voxel_graph)

### 3.2 Write / Read

In [9]:
# Write
vms.write_to_json_gz('voxel_maize_segmentation.json')

In [10]:
# Load
vms = phm_obj.VoxelSegmentation.read_from_json_gz('voxel_maize_segmentation.json')

### 3.3 Display

phm_display_notebook.show_segmentation(vms)

## 4. Maize Analysis

### 4.1 Process

In [11]:
vmsi = phm_seg.maize_analysis(vms)

### 4.2 How to save analysis result :

In [12]:
vmsi.write_to_json_gz('voxel_maize_segmentation_info.json.gz')

In [13]:
vmsi = phm_obj.VoxelSegmentation.read_from_json_gz('voxel_maize_segmentation_info.json.gz')

### 4.3 Visualize results: 

In [14]:
phm_display_notebook.show_segmentation(vmsi, size_ratio=2)

A Jupyter Widget